Evaluation Space

In [1]:
import numpy as np
import pandas as pd
import sys
import os
import time
from sqlfluff.core import Linter
import re
from difflib import SequenceMatcher
from sqlrespcompare import run_compare_query_results

In [2]:
print(sys.path)

['/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/home/kengqiangmk/Projects/chat-to-database-chatbot/venv/lib/python3.11/site-packages']


In [3]:


# # Get the absolute path of the tools folder
project_dir = os.path.abspath("..")  # Assuming the eval folder is in the project folder
chat2dbchatbot_dir = os.path.join(project_dir, "chat2dbchatbot")

# # Add the tools directory to the system path
sys.path.append(chat2dbchatbot_dir)


In [4]:
from tools.tagsql import run_tag_pipeline
from tools.ragsql import run_rag_pipeline

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
eval_file_path = './data/custom_db_exp.csv'
df = pd.read_csv(eval_file_path)
df.head()

,sql_complexity,sql_complexity_description,sql_prompt,sql
0,basic SQL,basic SQL with a simple select statement,Show all invoices that were made during Christ...,SELECT * FROM invoice WHERE (TO_CHAR(invoice_d...
1,single join,"only one join (specify inner, outer, cross)",Find the top 3 artists by total albums,"SELECT AR.name, COUNT(AR.name) as TotalAlbums ..."
2,aggregation and single join,"aggregation functions (COUNT, SUM, AVG, MIN, M...",list classic composers and their corresponding...,"SELECT T.composer, AVG(T.milliseconds) from tr..."
3,basic SQL,basic SQL with a simple select statement,list all the countries our customers are from,SELECT distinct country from customer;
4,simple aggregation,basic SQL with a simple select statement,What is the total invoice amount by country?,"SELECT billing_country, SUM(total) from invoic..."


In [6]:

linter = Linter(dialect='postgres')
lint_result_tag = ""
lint_result_rag = ""
for i in range(len(df)):
    # start_time_tag = time.perf_counter()
    # gen_sql_tag = await run_tag_pipeline(df.iloc[i]['sql_prompt'], "OpenAI", 0.1, "query_synthesis")
    # end_time_tag = time.perf_counter()
    # latency_tag = (end_time_tag - start_time_tag)*1000 # convert seconds to milli seconds
    # lint_return_tag = linter.lint_string(gen_sql_tag)
    # if lint_return_tag.violations:
    #     for violation in lint_return_tag.violations:
    #         lint_result_tag += f"- {violation.description} (Line: {violation.line_no}, Pos: {violation.line_pos})\n"
    # else: 
    #     lint_result_tag = "no error"

    start_time_rag = time.perf_counter()
    gen_sql_rag = run_rag_pipeline(df.iloc[i]['sql_prompt'], "OpenAI", 0.1)
    end_time_rag = time.perf_counter()
    latency_rag = (end_time_rag - start_time_rag)*1000 # convert seconds to milli seconds
    print(gen_sql_rag)
    # lint_return_rag = linter.lint_string(gen_sql_rag)
    # if lint_return_rag.violations:
    #     for violation in lint_return_rag.violations:
    #         lint_result_rag += f"- {violation.description} (Line: {violation.line_no}, Pos: {violation.line_pos})\n"
    # else: 
    #     lint_result_rag = "no error"

    # compact_gen_sql_tag = re.sub(r"\s+", "", gen_sql_tag).lower()
    compact_gen_sql_rag = re.sub(r"\s+", "", gen_sql_rag).lower()
    compact_sql = re.sub(r"\s+", "", df.iloc[i]['sql']).lower()

    # print(compact_gen_sql_tag)
    print(compact_gen_sql_rag)
    print(compact_sql)
    print("\n")

    # df.loc[i, 'exp_gen_similarity_tag'] = SequenceMatcher(None, compact_sql, compact_gen_sql_tag).ratio()
    # df.loc[i, 'exp_gen_len_diff_tag'] = len(compact_gen_sql_tag) - len(compact_sql)
    # df.loc[i, 'is_exp_gen_resp_equal_tag'] = run_compare_query_results(compact_gen_sql_tag, compact_sql)
    # df.loc[i, 'gen_sql_tag'] = gen_sql_tag
    # df.loc[i, 'latency_tag'] = latency_tag

    # df.loc[i, 'lint_result_rag'] = lint_result_rag
    # df.loc[i, 'exp_gen_similarity_rag'] = SequenceMatcher(None, compact_sql, compact_gen_sql_rag).ratio()
    # df.loc[i, 'exp_gen_len_diff_rag'] = len(compact_gen_sql_rag) - len(compact_sql)
    # df.loc[i, 'is_exp_gen_resp_equal_rag'] = run_compare_query_results(compact_gen_sql_rag, compact_sql)
    # df.loc[i, 'gen_sql_rag'] = gen_sql_rag
    # df.loc[i, 'latency_rag'] = latency_rag
    # df.loc[i, 'lint_result_rag'] = lint_result_rag


df.head(2)


NameError: name 'run_rag_pipeline' is not defined

In [28]:
print(df.iloc[0]['gen_sql'])
print(df.iloc[0]['lint_result'])
print(df['is_exp_gen_resp_equal'].tolist())

SELECT invoice_id, customer_id, invoice_date, total 
FROM invoice 
WHERE invoice_date::date IN ('2022-12-24', '2022-12-25', '2022-12-26', '2022-12-31', '2023-01-01')
ORDER BY invoice_date;
- Select targets should be on a new line unless there is only one select target. (Line: 1, Pos: 1)
- Unnecessary trailing whitespace. (Line: 1, Pos: 52)
- Unnecessary trailing whitespace. (Line: 2, Pos: 13)
- Line is too long (98 > 80). (Line: 3, Pos: 1)
- Files must end with a single trailing newline. (Line: 4, Pos: 22)

[False, False, False, False, False, False, False, False, False]


In [ ]:
print(df.iloc[3]['gen_sql'])
print(df.iloc[3]['lint_result'])